<a href="https://colab.research.google.com/github/nrohrbach/sharedmobility_monitoring/blob/main/sharedmobility_monitoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monitoring GBFS-Feeds on sharemobility.ch

Beschreibung...

In [203]:
pip install datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 9.6 MB/s eta 0:00:00


In [204]:
import requests
import pandas as pd
from datetime import datetime

In [3]:
# GBFS Feeds abfragen
url = 'https://gbfs.prod.sharedmobility.ch/v2/gbfs'
feeds = requests.get(url).json()

In [224]:
# Alle GBFS Feeds als Dataframe laden
id = [s['id'] for s in feeds['systems']]
url = [s['url'] for s in feeds['systems']]
gbfsfeeds = pd.DataFrame(list(zip(id, url)),columns =['provider', 'gbfsurl'])
providers = list(gbfsfeeds['provider'])
providers

['donkey_thun',
 'sponticar',
 'carvelo2go',
 'bird-kloten',
 'lime_winterthur',
 'lime_opfikon',
 'bolt_zurich',
 'bird-platform-partner-jmfleets-bulle',
 'pickebike_basel',
 'donkey_neuchatel',
 'lime_uster',
 'nextbike_ch',
 'pickebike_aubonne',
 'lime_basel',
 'lime_wetzikon',
 'bird-basel',
 'donkey_freiburg',
 'donkey_kreuzlingen',
 'pickebike_fribourg',
 'donkey_ge',
 'donkey_li',
 'bird-platform-partner-jmfleetswl-biel',
 'bird-zurich',
 'donkey_le_locle',
 'donkey_sion',
 'share_birrer_ch',
 'donkey_yverdon-les-bains',
 'velospot',
 '2em_cars',
 'mobility',
 'edrivecarsharing']

## Auswertung pro Provider

In [218]:
# Auswertung station_status.json
providerList = []
VehiclesInStationList = []
NumberOfStationsList = []

for provider in providers:
  try:
      stationstatus = requests.get('https://gbfs.prod.sharedmobility.ch/v2/gbfs/' + provider +'/station_status').json()
      stationstatus = [s['num_bikes_available'] for s in stationstatus['data']['stations']]
      VehiclesInStationList.append(sum(stationstatus))
      NumberOfStationsList.append(len(stationstatus))
      providerList.append(provider)
  except:
      VehiclesInStationList.append('nan')
      NumberOfStationsList.append('nan')
      providerList.append(provider)

# Dataframe erstellen
dict = {'Provider': providerList, 'NumberOfStations': NumberOfStationsList, 'VehiclesInStation': VehiclesInStationList}
dfStationStatus = pd.DataFrame(dict)


In [222]:
# Auswertung free_bike_status.json
providerList = []
FreeBikeList = []

for provider in providers:
  try:
      freebikesstatus = requests.get('https://gbfs.prod.sharedmobility.ch/v2/gbfs/' + provider +'/free_bike_status').json()
      freebikesstatus = [s['bike_id'] for s in freebikesstatus['data']['bikes']]
      FreeBikeList.append(len(freebikesstatus))
      providerList.append(provider)
  except:
      FreeBikeList.append('nan')
      providerList.append(provider)

# Dataframe erstellen
dict = {'Provider': providerList, 'NumberOfFreeBikes': FreeBikeList}
dfFreeBikes = pd.DataFrame(dict)

In [223]:
# Dataframe zusammenführen und speichern
dfMonitoring = pd.merge(dfStationStatus, dfFreeBikes, how="left", on=["Provider"])
dfMonitoring['Date'] = datetime.today().strftime("%Y-%m-%d")

#Speichern
dfMonitoring.to_csv("Sharedmobility_Providers.csv", header=False, index=False, mode='a')
dfMonitoring

,Provider,NumberOfStations,VehiclesInStation,NumberOfFreeBikes,Date
0,donkey_thun,38,166,0,2023-07-19
1,sponticar,nan,nan,0,2023-07-19
2,carvelo2go,nan,nan,0,2023-07-19
3,bird-kloten,nan,nan,50,2023-07-19
4,lime_winterthur,1,144,143,2023-07-19
5,lime_opfikon,1,75,76,2023-07-19
6,bolt_zurich,nan,nan,704,2023-07-19
7,bird-platform-partner-jmfleets-bulle,0,0,144,2023-07-19
8,pickebike_basel,60,25,367,2023-07-19
9,donkey_neuchatel,56,242,0,2023-07-19


In [271]:
# Validieren!
providerList = []
HasErrors = []
NumberOfErrors = []

for provider in providers:
    try:
      headers = {
          'accept': '*/*',
          'Content-Type': 'application/json',
      }

      json_data = {
          'url': 'https://gbfs.prod.sharedmobility.ch/v2/gbfs/'+ provider +'/gbfs',
          'options': {
              'freefloating': False,
              'docked': False,
              'version': None,
              'auth': {
                  'type': None,
                  'basicAuth': {
                      'user': None,
                      'password': None,
                  },
                  'bearerToken': {
                      'token': None,
                  },
                  'oauthClientCredentialsGrant': {
                      'user': None,
                      'password': None,
                      'tokenUrl': None,
                  },
              },
          },
      }
      response = requests.post('https://gbfs-validator.netlify.app/.netlify/functions/validator', headers=headers, json=json_data).json()

      providerList.append(provider)
      HasErrors.append(response['summary']['hasErrors'])
      NumberOfErrors.append(response['summary']['errorsCount'])

    except:
      providerList.append(provider)
      HasErrors.append('nan')
      NumberOfErrors.append('nan')

In [275]:
len(NumberOfErrors)

31

In [276]:
# Dataframe erstellen
dict = {'Provider': providerList, 'Fehlerhaft': HasErrors, 'AnzahlFehler': NumberOfErrors}
dfValidate = pd.DataFrame(dict)

ValueError: ignored